## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}
    
        


# normalize datetime
# min_date = min(dates)
# days = [(d-min_date).total_seconds() / (60*60*24) for d in dates]
# mean_days, std_days = np.mean(days), np.std(days)
# def normalize_date(min_date, mean_days, std_days, date):
#     days = (date-min_date).total_seconds() / (60*60*24)
#     return (days-mean_days)/std_days
# with tqdm(total=len(u_map)) as pbar:
#     for _,v in u_map.items():
#         for i, pair in enumerate(v):
#             date, food = pair
#             v[i] = (normalize_date(min_date, mean_days, std_days, date), food)
#         pbar.update(1)
        
# item-user matrix, for ALS




100%|██████████| 2681494/2681494 [00:20<00:00, 129751.78it/s]


In [2]:
import numpy as np
a = []
b = []

for k,v in u_map.items():
    ds = [d for d,f in v]
    
    set_ds = set(ds)
    a.append(len(set_ds))
    b.append(len(ds))
print max(a), min(a),np.mean(a), np.std(a)
print max(b), min(b),np.mean(b), np.std(b)

160 23 95.21165644171779 34.48409857182605
5569 194 1028.1802147239264 599.6642539977294


### ALS embedding

In [3]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix


with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    Y_map = {}
    pairs = []
    history_map = {} # for training and validation
    for user,v in u_map.items():
        history = v
        dates_history = [d for d,f in history]
        min_date, max_date = min(dates_history), max(dates_history)
        date_list = sorted(set(dates_history))
        rest_date_set = set(date_list[-len(date_list)//10:])
        
        
        
        history_map[k] = [d for d,f in history if d not in rest_date_set]
#         calculate the Y
        y = np.zeros([1, len(food_map)])
        for d,f in history:
            if d in rest_date_set:
                y[0, food_map[f]] = 1
            else:
                pair = (food_map[f], user_map[user])
                pairs.append(pair)
        Y_map[user] = y
        pbar.update(1)

        
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        f, u = pair
        rows.append(f)
        cols.append(u)
    data = np.ones([len(rows),])
    item_user_matrix = csr_matrix((data, (rows,cols)), shape=(len(food_map), len(user_map)))
    
print len(Y_map)    
        
    



100%|██████████| 2608/2608 [00:01<00:00, 1505.27it/s]

2608


In [4]:
import implicit
import os

os.environ["OPENBLAS_NUM_THREADS"] = '1'
als_model = implicit.als.AlternatingLeastSquares(factors=128, iterations=100)
als_model.fit(item_user_matrix)
print len(rows)




100%|██████████| 100.0/100 [00:28<00:00,  3.51it/s]

492527


### Transformer with ALS embedding Training

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import Transformer
import numpy as np

class Net(nn.Module):

    def __init__(self, dm, p_drop):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p_drop)
        
        self.transformer = Transformer.Transformer(2, dm, dm, dm, 8, p_drop, d_ff=dm*4)
        self.summary_weight = nn.Parameter(torch.FloatTensor(1, dm))
        torch.nn.init.xavier_uniform_(self.summary_weight)
        
        self.output_linear = nn.Linear(dm, 1)

    def forward(self, food, history):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        Q = food
        K = history
        
        batch, K_len, d = K.size()
        Q = self.summary_weight.repeat(batch,1).view(batch, 1, -1)
        batch, Q_len, d = Q.size()
        Q_mask_len = np.ones([batch])
        de_out = self.transformer(Q, K, Q_mask_len)
        de_out = de_out.view(batch, d)
        
        out = self.output_linear(de_out)
        out = torch.sigmoid(out)
        return out

batch = 7
dm = 128
K = torch.rand([batch, 18, dm]).cuda()
Q = torch.rand([dm]).cuda()
model = Net(dm, 0.1).cuda()
print(Q.dtype)
o = model(Q, K)
# print t
print(o.size())
# print o

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))


criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

torch.float32
torch.Size([7, 1])
3694593


#### prepare sample generator


In [6]:
user_emb = als_model.user_factors
food_emb = als_model.item_factors
print user_emb.shape, food_emb.shape

val_num = len(u_map) // 10
idx = np.random.permutation(len(u_map))
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:u_map[k] for k in u_map.keys()[val_num:]}
val_u_map = {k:u_map[k] for k in u_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, u_map, food_map, Y_map, food_emb, max_history_len):
    G = boostrap_generator(u_map, food_map, Y_map, food_emb, max_history_len)
    while True:
        X = []
        pos_Y = []
        neg_Y = []
        for i in range(batch_size):
            x, pos_y, neg_y = next(G)
            x = np.pad(x, ((0,max_history_len-x.shape[0]),(0,0)), 'constant', constant_values=0)
            X.append(np.expand_dims(x, axis=0))
            pos_Y.append(np.expand_dims(pos_y, axis=0))
            neg_Y.append(np.expand_dims(neg_y, axis=0))
        yield np.vstack(X), np.vstack(pos_Y), np.vstack(neg_Y) 
def boostrap_generator(u_map, food_map, Y_map, food_emb, max_history_len):
    while True:
        keys = u_map.keys()
        for user_idx in np.random.permutation(len(u_map)):
            user = keys[user_idx]
            X = []
            Y = Y_map[user].flatten()
            history = u_map[user]
            ds = [d for d,f in history]
            fs = [f for d,f in history]
            for idx in np.argsort(ds):
                food = fs[idx]
                x = food_emb[food_map[food], :]
                X.append(np.expand_dims(x, axis=0))
            X = np.vstack(X)
            
    #         positive sample
            idx = np.random.permutation(len(Y))
            pos_i = neg_i = -1
            for i in idx:
                if Y[i] == 1 and pos_i == -1:
                    pos_i = i
                if Y[i] == 0 and neg_i == -1:
                    neg_i = i
                if pos_i != -1 and neg_i != -1:
                    break
            yield X, food_emb[pos_i], food_emb[neg_i]
            
    

G = batch_boostrap_generator(32, train_u_map, food_map, Y_map, food_emb, max_history_len=5600)
val_G = batch_boostrap_generator(32//2, val_u_map, food_map, Y_map, food_emb, max_history_len=5600)

x, pos_y, neg_y = next(G)
print x.shape, pos_y.shape, neg_y.shape
x, pos_y, neg_y = next(val_G)
print x.shape, pos_y.shape, neg_y.shape

G = boostrap_generator(train_u_map, food_map, Y_map, food_emb, max_history_len=5600)
x, pos_y, neg_y = next(G)
print x.shape, pos_y.shape, neg_y.shape

(2608, 128) (5532, 128)
(32, 5600, 128) (32, 128) (32, 128)
(16, 5600, 128) (16, 128) (16, 128)
(498, 128) (128,) (128,)


In [9]:
from collections import deque
from tqdm import tqdm as tqdm
import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 100 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
        
acc_q = deque(maxlen=1000)
loss_q = deque(maxlen=1000)
val_acc_q = deque(maxlen=1000)
val_loss_q = deque(maxlen=1000)
t = time.time()
best_acc  = 0
best_loss = float('inf')

epochs = 100
batch_size = 1
G = boostrap_generator(train_u_map, food_map, Y_map, food_emb, max_history_len=5600)
val_G = boostrap_generator(val_u_map, food_map, Y_map, food_emb, max_history_len=5600)

print 'start training.'
with open('log.txt', 'a') as f:
    for e in range(epochs):
        iters = len(food_map) * len(user_map)
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                x, pos_y, neg_y = next(G)
                x, pos_y, neg_y = np.expand_dims(x, axis=0),np.expand_dims(pos_y, axis=0),np.expand_dims(neg_y, axis=0)
                x = torch.FloatTensor(x).cuda()
                pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
                
                if np.random.rand(1) > 0.5:
#                 positive
                    a=output = model(pos_y.cuda(), x)
                    pred = output.flatten() > 0.5
                    label = torch.ones_like(output).cuda()
                    pos_loss = criterion(output, label)

                    acc = torch.sum(pred == label.type(torch.uint8)) / float(output.shape[0])
                    acc_q.append(acc)
                    pos_loss.backward()
                    optimizer.step()
                else:

#                 negative
                    a=output = model(neg_y.cuda(), x)
                    pred = output.flatten() > 0.5
                    label = torch.zeros_like(output).cuda()
                    neg_loss = criterion(output, label )
                    acc = torch.sum(pred == label.type(torch.uint8)) / float(output.shape[0])
                    acc_q.append(acc)
                    neg_loss.backward()
#                     loss = pos_loss + neg_loss
#                     loss.backward()
                    optimizer.step()
                with torch.no_grad():
                    model.eval()
#                 positive
                    x, pos_y, neg_y = next(val_G)
                    x, pos_y, neg_y = np.expand_dims(x, axis=0),np.expand_dims(pos_y, axis=0),np.expand_dims(neg_y, axis=0)
                    x = torch.FloatTensor(x).cuda()
                    pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
#                       
                    b=output = model(pos_y.cuda(), x)
                    pred = output.flatten() > 0.5
                    label = torch.ones_like(output).cuda()
                
                    pos_loss = criterion(output, label )
                
                    val_acc = torch.sum(pred == label.type(torch.uint8)) / float(output.shape[0])
                    val_acc_q.append(val_acc)

#                 negative
                    c=output = model(neg_y.cuda(), x)
                    pred = output.flatten() > 0.5
                    label = torch.zeros_like(output).cuda()
        
                    neg_loss = criterion(output, label )
                    val_acc = torch.sum(pred == label.type(torch.uint8)) / float(output.shape[0])
                    val_acc_q.append(val_acc)
                    
                    val_loss = pos_loss + neg_loss
                acc = np.mean(acc_q)
                val_acc = np.mean(val_acc_q)

#                 dump_log(model, it+1, loss, acc, val_loss, val_acc, f, './tmp.pt')

                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.3f' % (acc, val_acc,a,b,c), refresh=False)
                pbar.update(1)

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/14427456 [00:00<?, ?it/s]

start training.


  0%|          | 346/14427456 [01:04<1086:38:02,  3.69it/s, acc : 0.488, val_acc : 0.500, 0.578, 0.574, 0.574]


RuntimeError: CUDA error: out of memory

In [11]:
import torch
a = torch.tensor([[0.49]])
b = torch.zeros_like(a)
print a.shape
print a > 0.5
c = a > 0.5
print b,c
print b.type(torch.uint8) == c


torch.Size([1, 1])
tensor([[0]], dtype=torch.uint8)
tensor([[0.]]) tensor([[0]], dtype=torch.uint8)
tensor([[1]], dtype=torch.uint8)


In [ ]:
# from collections import deque
# import time
# def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
#     log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
#     log_file_stream.write(log_text)
#     if n_iter % 100 == 0 :
#         log_file_stream.flush()
#         torch.save(model, tmp_model_path)
        
# acc_q = deque(maxlen=1000)
# loss_q = deque(maxlen=1000)
# val_acc_q = deque(maxlen=1000)
# val_loss_q = deque(maxlen=1000)
# t = time.time()
# best_acc  = 0
# best_loss = float('inf')

# epochs = 100
# batch_size = 1
# G = batch_boostrap_generator(batch_size, train_u_map, food_map, Y_map, food_emb, max_history_len=5600)
# val_G = batch_boostrap_generator(batch_size//2, val_u_map, food_map, Y_map, food_emb, max_history_len=5600)

# print 'start training.'
# with open('log.txt', 'a') as f:
#     for e in range(epochs):
#         iters = len(food_map) * len(user_map)
#         for it in range(iters):
#             with tqdm(total=iters) as pbar:

#                 optimizer.zero_grad()
#                 model.train()
#                 x, pos_y, neg_y = next(G)
#                 x = torch.FloatTensor(x).cuda()
#                 pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
# #                 positive
#                 output = model(pos_y.cuda(), x)
#                 label = torch.ones_like(output).cuda()
#                 pos_loss = criterion(output, label )
#                 acc = torch.sum(output.flatten() == label) / float(output.shape[0])
#                 acc_q.append(acc)

# #                 negative
#                 output = model(neg_y.cuda(), x)
#                 label = torch.zeros_like(output).cuda()
#                 neg_loss = criterion(output, label )
#                 acc = torch.sum(output.flatten() == label) / float(output.shape[0])
#                 acc_q.append(acc)

#                 loss = pos_loss + neg_loss
#                 loss.backward()
#                 optimizer.step()
#                 with torch.no_grad:
#                     model.eval()
# #                 positive
#                     x, pos_y, neg_y = next(val_G)
#                     output = model(pos_y.cuda(), x)
#                     label = torch.ones_like(output).cuda()
#                     pos_loss = criterion(output, label )
                
#                     val_acc = torch.sum(output.flatten() == label) / float(output.shape[0])
#                     val_acc_q.append(val_acc)

# #                 negative
#                     output = model(neg_y.cuda(), x)
#                     label = torch.zeros_like(output).cuda()
#                     neg_loss = criterion(output, label )
#                     val_acc = torch.sum(output.flatten() == label) / float(output.shape[0])
#                     val_acc_q.append(val_acc)
                    
#                     val_loss = pos_loss + neg_loss
#                 acc = np.mean(acc_q)
#                 val_acc = np.mean(val_acc_q)

#                 dump_log(model, it+1, loss, acc, val_loss, val_acc, f, './tmp.pt')

#                 pbar.set_postfix_str('acc : %.3f, val_acc : %.3f' % (acc, val_acc), refresh=False)
#                 pbar.update(1)

# # Train model
# print("Optimization Finished!")
# # print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

In [ ]:
salnj;kvahjk
# print u_map[6]
buf  = [] 
def ck(ds):
    l = max(ds) - min(ds)
    min_d = min(ds)
    
#     d_list = [(d-min_d) for d in ds]
#     d_list = sorted(set(d_list))
    d_list = sorted(set(ds))
#     print l, len(d_list)
    for i,d in enumerate(d_list):
        if i == 0:
            continue
        d_ = d_list[i-1]
        if (d - d_).days.real != 1:
            d_diff = (d - d_).days.real
            buf.append(d_diff)
#             print d_diff
#             assert d_diff < 10
def cf(ds):
    l = max(ds) - min(ds)
    min_d = min(ds)
    
#     d_list = [(d-min_d) for d in ds]
#     d_list = sorted(set(d_list))
    d_list = sorted(set(ds))
    
#     print l, len(d_list)
    for i,d in enumerate(d_list):
        if i == 0:
            continue
        d_ = d_list[i-1]
        if (d - d_).days.real != 1:
            d_diff = (d - d_).days.real
            buf.append(d_diff)
#             print d_diff
#             assert d_diff < 10
                        
    
# for i in [6,8,12]:
with tqdm(total=len(u_map)) as pbar:
    buf = []
    buf2 = []
    for i in u_map.keys():
        ds = [d for d,f in u_map[i]]
        threshold = ((max(ds) - min(ds)).days.real * (4./5.))
        min_d = min(ds)
        fl = []
        m = {}
        for d,f in u_map[i]:
            if (d - min_d).days.real > threshold:
                fl.append(f)
            if d not in m:
                m[d] = []
            m[d].append(f)
        for k,v in m.items():    
            buf2.append(len(set(v)))
#         print len(u_map[i]), len(fl), len(set(fl))
        buf.append(len(set(fl)))
        #     print i, min(ds), max(ds), len(ds)
#         ck(ds)
        pbar.update(1)
# d6 = [d for d,f in u_map[6]]
# ck(d6)
    
#     print ck(ds)
# print u_map[33]
print np.mean(buf), np.std(buf)
print np.mean(buf2), np.std(buf2)


In [ ]:


print min(dates), max(dates),  max(dates)- min(dates)


In [ ]:
print min(dates)
m = min(dates)
print max(dates)
a = dates[0]
print dir(a - min(dates))
print a
# def normalize_date(min_date, max_date, date):
# for k in u_map.keys()[:10]:
#     print len(u_map[k])
ds = [(d-m).total_seconds() / (60*60*24) for d in dates]
# import numpy as np
print np.mean(ds)
print np.min(ds), np.max(ds)

In [ ]:
import keras
from os.path import join
import os
from bs4 import BeautifulSoup as BS
from constants import MAX_SEQUENCE_LENGTH, MAX_NUM_WORDS, EMBEDDING_DIM
from keras.preprocessing.sequence import pad_sequences
import numpy as np
np.random.seed(1337)
def quote_title_abstract(xml_path):
    with open(xml_path, 'r') as f:
        data = f.read()
    soup = BS(data)
    title, abstract = soup.find('title').text, soup.find('abstract').text
    return title.strip(), abstract.strip()

# text preprocessing
data_path = join('./','kaggle/')
xml_dir = join(data_path, 't2-doc')
xml_list = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
# print(len(xml_list))


texts = []

for xml in xml_list:
    path = join(xml_dir,xml)
    title, abstract = quote_title_abstract(path)
    text = title + '' + abstract
    texts.append(text)
#     texts.append(title)
#     texts.append(abstract)
print('read all %d xml files.' % len(xml_list))
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=True, split=' ', char_level=False, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
xml_id_map = {}
for i,xml in enumerate(xml_list):
    node_id = int(xml.replace('.xml',''))
    xml_id_map[node_id] = data[i,:]


print('Preparing embedding matrix.')
embeddings_index = {}
# with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r', encoding='utf8') as f:
with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('done')